# Term Project II

The main task is to build a personal food recipe recommendation web-application. The followings are its main features

- The user can search for recipe by name.
- The user can search for recipe by ingredients.
- The user receive the suggestion recipes she/he maybe interested in.

In [15]:
import pandas as pd

from sklearn import preprocessing
from sklearn import neighbors
from sklearn import tree
from sklearn import naive_bayes
from sklearn import model_selection
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy import sparse

import numpy as np
from numpy import hstack

import multiprocessing as mp

import string

import json
import re

import nltk
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

from spellchecker import SpellChecker
import os
from pathlib import Path

## Sample data

In [8]:
food = pd.read_csv('food_ingredients.csv')
food.drop('Unnamed: 0',axis='columns',inplace=True)
food.drop('Ingredients',axis='columns',inplace=True)
food.head()

,Title,Instructions,Image_Name,Cleaned_Ingredients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,Crispy Salt and Pepper Potatoes,Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,Thanksgiving Mac and Cheese,Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,Italian Sausage and Bread Stuffing,Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,Newton's Law,Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."


In [9]:
title = food.iloc[0]['Title']
print(title)

Miso-Butter Roast Chicken With Acorn Squash Panzanella


In [96]:
check = food.iloc[0]["Cleaned_Ingredients"].split(',')
check[0] = check[0].replace('[','')
check[-1] = check[-1].replace(']','')
for i in check :
    print(i)

'1 (3½–4-lb.) whole chicken'
 '2¾ tsp. kosher salt
 divided
 plus more'
 '2 small acorn squash (about 3 lb. total)'
 '2 Tbsp. finely chopped sage'
 '1 Tbsp. finely chopped rosemary'
 '6 Tbsp. unsalted butter
 melted
 plus 3 Tbsp. room temperature'
 '¼ tsp. ground allspice'
 'Pinch of crushed red pepper flakes'
 'Freshly ground black pepper'
 '⅓ loaf good-quality sturdy white bread
 torn into 1" pieces (about 2½ cups)'
 '2 medium apples (such as Gala or Pink Lady; about 14 oz. total)
 cored
 cut into 1" pieces'
 '2 Tbsp. extra-virgin olive oil'
 '½ small red onion
 thinly sliced'
 '3 Tbsp. apple cider vinegar'
 '1 Tbsp. white miso'
 '¼ cup all-purpose flour'
 '2 Tbsp. unsalted butter
 room temperature'
 '¼ cup dry white wine'
 '2 cups unsalted chicken broth'
 '2 tsp. white miso'
 'Kosher salt'
 'freshly ground pepper'


In [11]:
instru = ingre = food.iloc[0]['Instructions'].split('.')
for i in instru :
    print(i)

Pat chicken dry with paper towels, season all over with 2 tsp
 salt, and tie legs together with kitchen twine
 Let sit at room temperature 1 hour

Meanwhile, halve squash and scoop out seeds
 Run a vegetable peeler along ridges of squash halves to remove skin
 Cut each half into ½"-thick wedges; arrange on a rimmed baking sheet

Combine sage, rosemary, and 6 Tbsp
 melted butter in a large bowl; pour half of mixture over squash on baking sheet
 Sprinkle squash with allspice, red pepper flakes, and ½ tsp
 salt and season with black pepper; toss to coat

Add bread, apples, oil, and ¼ tsp
 salt to remaining herb butter in bowl; season with black pepper and toss to combine
 Set aside

Place onion and vinegar in a small bowl; season with salt and toss to coat
 Let sit, tossing occasionally, until ready to serve

Place a rack in middle and lower third of oven; preheat to 425°F
 Mix miso and 3 Tbsp
 room-temperature butter in a small bowl until smooth
 Pat chicken dry with paper towels, then r

In [12]:
image_name = food.iloc[0]['Image_Name']
print(image_name)

miso-butter-roast-chicken-acorn-squash-panzanella


![somehting](./food_image/miso-butter-roast-chicken-acorn-squash-panzanella.JPG)

# Spell Checker Feature

In [13]:
context = Path("eng-simple_wikipedia_2021_300K-sentences.txt").read_text("utf-8")
context = re.sub('[^A-Za-z]'," ",context)
context = " ".join(context.split())
context = context.lower()

list_of_word = context.split(" ")

In [14]:
spell = SpellChecker()

spell.word_frequency.load_words(list_of_word)

word = "Miso Buter Roats Chickken Wiht Acon Squah Panzanella"

misspelled = spell.unknown(word.split(" "))

if len(misspelled) != 0 :
    for word in misspelled:
        print(spell.correction(word))
        candidate_list = [i for i in spell.candidates(word)]
        print(candidate_list)
#         print(spell.candidates(word))
else:
    print("no mispelling")

con
['aron', 'anon', 'acorn', 'ason', 'axon', 'alon', 'acou', 'bacon', 'econ', 'acdn', 'con', 'icon', 'amon', 'aon', 'aco', 'avon', 'lacon', 'apon', 'acton', 'acun', 'aeon', 'afon', 'akon', 'macon']
roads
['oats', 'goats', 'boats', 'rots', 'roads', 'roams', 'croats', 'moats', 'roat', 'doats', 'coats', 'roast', 'roots', 'roars', 'groats', 'rotas', 'roasts', 'rats', 'routs']
chicken
['chicken']
outer
['bute', 'buler', 'cuter', 'bunter', 'beter', 'buyer', 'butler', 'puter', 'biter', 'buner', 'butter', 'buster', 'bater', 'muter', 'outer', 'guter', 'butser']
manzanilla
['manzanilla']
squad
['squad', 'squaw', 'squab', 'squat', 'squash', 'shuah']


# Search Feature

# Prepre Search Feature

- tf-idf
- bm25
- elastic search

In [17]:
vectorizer = TfidfVectorizer(ngram_range=(1,3))
X = vectorizer.fit_transform(food["Title"].astype('U'))

## Prepare TF-IDF

In [47]:
def serach_by_tf_idf(query):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X,query_vec).reshape((-1,))
    return results.argsort()[-50:][::-1]

## Prepare BM25

In [26]:
class BM25(object):
    def __init__(self, b=0.75, k1=1.6):
        self.vectorizer = TfidfVectorizer(norm=None, smooth_idf=False,ngram_range=(1,3))
        self.b = b
        self.k1 = k1

    def fit(self, X):
        self.vectorizer.fit(X)
        y = super(TfidfVectorizer, self.vectorizer).transform(X)
        self.avdl = y.sum(1).mean()

    def transform(self, q, X):
        b, k1, avdl = self.b, self.k1, self.avdl

        X = super(TfidfVectorizer, self.vectorizer).transform(X)
        len_X = X.sum(1).A1
        q, = super(TfidfVectorizer, self.vectorizer).transform([q])
        assert sparse.isspmatrix_csr(q)

        X = X.tocsc()[:, q.indices]
        denom = X + (k1 * (1 - b + b * len_X / avdl))[:, None]
        idf = self.vectorizer._tfidf.idf_[None, q.indices] - 1.
        numer = X.multiply(np.broadcast_to(idf, X.shape)) * (k1 + 1)                                                          
        return (numer / denom).sum(1).A1

In [27]:
bm25 = BM25()
bm25.fit(food["Title"].astype('U'))

In [65]:
def search_by_bm25(query):
    result = bm25.transform(query,food["Title"].astype('U'))
    return result.argsort()[-50:][::-1]

## Search by Food Title ( by name) Feature

In [56]:
for i in serach_by_tf_idf("sweet english tea") :
    print(food.iloc[i]['Title'])

Sweet Tea
English Cookies
English Pancakes
Sweet Peach Tea
Mint Tea
Ginger Tea
Beef Tea
Tea Cakes
Tipsy Tea with Homemade Sweet Tea
English Chili Sauce
English Pea Hummus
Cleansing Tea
English Peas with Mint
English Jam Bag Pudding
Superb English Plum Pudding
Green Tea Cheesecake
Pasta Carbonara with English Peas
Green Tea Margarita
Green-Tea Truffles
Apple Tea Cakes
Moroccan Mint Tea
Grilled Tea-Brined Turkey with Tea-and-Lemon Gravy
Moroccan Tea Biscuits
Sweet Tea with Vodka and Lemonade
Hibiscus Tea Sorbet
Pimm's Iced Tea
Sorrel (Hibiscus) Tea
Tea-Totaler’s Toddy
Sweet Walnuts
Tea-and-Honey Crisps
Thai-Style Iced Tea
Lavender Honey Tea Bread
Cold-Brew Iced Tea
Bourbon Fruit Tea Punch
Swirled Sesame Tea Cake
Sweet Crêpes
Golden Milk Turmeric Tea
Kumquat-Cardamom Tea Bread
Sweet Palmiers
Sweet Blini
Sweet Pretzels
Ultimate Hydrating Iced Tea
Sweet Amigos
Classic Bubble Tea Recipe
Butterscotch Puddings with Whipped Cream and Crushed English Toffee
Gluten-Free Chocolate Tea Cake
Yuba Pa

In [58]:
for i in search_by_bm25("taco kabub") :
    print(food.iloc[i]['Title'])

Fish Taco Platter
Guacamole Taquero: Taco-Shop Guacamole
Guajillo-Braised Beef Short Rib Taco
Butternut Squash, Kale, and Crunchy Pepitas Taco
Mushroom, Rajas, and Corn Taco with Queso Fresco
Fresh Taco Salad with Creamy Avocado-Lime Dressing
Breakfast Taco Hobo Packs with Black Beans, Zucchini, and Corn
Instant Pot Red Bean and Quinoa Soup with Taco Fixins
Turkey Leg Confit Taco with Pasilla Purée and Pickled Peach Salsa
Charred Octopus Taco with Harissa, Chorizo Crushed Potatoes, and Pickled Ramps
Apricot-Pistachio Muffins Baked on the Grill
Tarragon-Cilantro Salsa Verde
Mexican Poblano, Spinach, and Black Bean "Lasagne" with Goat Cheese
Shellfish Mixed Grill
Grilled Zucchini Bread With Mascarpone and Berries
Linguine with Grilled Tuna, Capers and Parsley
Sambal-Orange Vinaigrette
Pasta with Grilled Tomato and Scallion Sauce
Grilled Leg of Lamb with Herb Salt
Red, White, and Blueberry Pie
Tomato, Onion, and Roasted Lemon Salad
Pasta with Grilled Sausage, Peppers and Eggplant
Rice Sal

## Search By Ingredient Feature

### Check answer

In [94]:
ingre = food[food.Title == "Crock Pot Ribs"]["Cleaned_Ingredients"]
print(ingre)

3883    ['1 3–4 pound country style pork ribs, also kn...
Name: Cleaned_Ingredients, dtype: object


In [98]:
check = food.iloc[3883]["Cleaned_Ingredients"].split(',')
check[0] = check[0].replace('[','')
check[-1] = check[-1].replace(']','')
for i in check :
    print(i)

'1 3–4 pound country style pork ribs
 also known as baby back ribs (about 2 racks)'
 '1 teaspoon salt'
 '1 cup bbq sauce plus additional for serving'


In [66]:
bm25_ingr = BM25()
bm25_ingr.fit(food["Cleaned_Ingredients"].astype('U'))

In [67]:
vectorizer = TfidfVectorizer(ngram_range=(1,3))
X_ingr = vectorizer.fit_transform(food["Cleaned_Ingredients"].astype('U'))

In [68]:
def serach_ingre_tf_idf(query):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X_ingr,query_vec).reshape((-1,))
    return results.argsort()[-50:][::-1]

In [69]:
def search_ingre_bm25(query):
    result = bm25.transform(query,food["Cleaned_Ingredients"].astype('U'))
    return result.argsort()[-50:][::-1]

In [73]:
for i in serach_ingre_tf_idf("pork") :
    print(food.iloc[i]['Title'])

Cassoulet in the Style of Toulouse (Cassoulet de Toulouse)
James Beard's Roasted Spareribs
Porchetta
Carnitas: Braised and Fried Pork
Slow-Cooked Carnitas Tacos
Double-Pork Carnitas
Pork Shoulder With Pineapple and Sesame Broccoli
Pork Conserva with Green Tomato Agrodolce
Smoked Pork in Pastry
Grilled Pork Shoulder With Butter Vinegar Sauce
Grilled Pork Tenderloin with Fresh Fig Skewers
Homemade Bratwurst Bites with Beer and Horseradish Mustard
Grilled Pork Chops with Garlic Lime Sauce
Brown Sugar BBQ Pork Butt
Porchetta With Roasted Potatoes
Pecan-Crusted Pork Tenderloin Pinwheels with Carolina Mustard Sauce
Country Terrine
Roast Pork Loin With Rosemary and Garlic
Tacos of Carnitas Roasted with Orange, Milk, and Pepper
Crown Roast of Pork with Onion and Bread-Crumb Stuffing
Vietnamese Caramelized Grilled Pork
Milk Braised Pork Shoulder with Semolina Gnocchi
Rosemary Pork Chops
Chicharrón Casera
Smoked Scotch Eggs
Five Spice Pork Rillettes
Pork Shoulder Steaks with Grilled Mustard Gree

In [71]:
for i in search_ingre_bm25("pork") :
    print(food.iloc[i]['Title'])

James Beard's Roasted Spareribs
Chicharrón Casera
Dan Barber's Pork Belly
Porchetta
Grilled Pork Tenderloin with Molasses and Mustard
Carnitas: Braised and Fried Pork
Pecan-Crusted Pork Tenderloin Pinwheels with Carolina Mustard Sauce
Bacon-Molasses Breakfast Sausage
Tacos of Carnitas Roasted with Orange, Milk, and Pepper
Grilled Pork Shoulder With Butter Vinegar Sauce
Pork Shoulder With Pineapple and Sesame Broccoli
Caramelized Pork Rinds
Sausage Rolls
Pineapple Pork Chops
Tortillas de Harina (Flour Tortillas)
Pork Conserva with Green Tomato Agrodolce
Brown Sugar BBQ Pork Butt
Smoked Scotch Eggs
Sautéed Pork Tenderloin with Prunes
Grilled Pork Tenderloin with Fresh Fig Skewers
Vietnamese Caramelized Grilled Pork
Rosemary Pork Chops
Bacon-Wrapped Pig Wings
Cassoulet in the Style of Toulouse (Cassoulet de Toulouse)
Porchetta With Roasted Potatoes
Slow-Cooked Carnitas Tacos
Beer-Braised Carnitas
Weeknight Porchetta
Crock Pot Ribs
Crock Pot Pulled Pork Tacos
Pork Tenderloin with Apricot M

# The Suggestion System using machine learning